In [1]:
import numpy as np

from live_strat import RSIRisingFalling
from datetime import datetime

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [26]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=4500,
)
datetimes = candles[:, CandleBodyType.Timestamp].astype("datetime64[ms]")
open_prices = candles[:, CandleBodyType.Open]
high_prices = candles[:, CandleBodyType.High]
low_prices = candles[:, CandleBodyType.Low]
closing_prices = candles[:, CandleBodyType.Close]

In [27]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(30, 61, 5),
)

In [28]:
backtest_settings = BacktestSettings(gains_pct_filter=0, total_trade_filter=10)

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)
equity = 1000
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=equity,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([1, 0.25]),
    sl_based_on_lookback=np.array([30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1, 2, 3]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [29]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 7
Total order settings to test: 24
Total combinations of settings to test: 168
Total candles: 4,500
Total candles to test: 756,000


In [30]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
116,6,6,57.0,30,27,449.319,52.632,0.827,1379.673,5493.191,4493.191
62,4,0,39.0,24,15,280.253,61.538,0.821,527.988,3802.529,2802.529
14,2,0,29.0,19,10,138.448,65.517,0.820,150.752,2384.484,1384.484
118,6,8,43.0,20,23,384.711,46.512,0.806,892.022,4847.114,3847.114
26,2,12,29.0,19,10,129.390,65.517,0.802,178.108,2293.902,1293.902
92,5,6,62.0,30,32,283.506,48.387,0.800,1143.118,3835.059,2835.059
106,5,20,31.0,13,18,540.001,41.935,0.797,736.569,6400.010,5400.010
15,2,1,22.0,14,8,193.846,63.636,0.796,170.169,2938.461,1938.461
119,6,9,40.0,19,21,291.835,47.500,0.794,773.996,3918.351,2918.351
120,6,10,36.0,17,19,287.574,47.222,0.794,701.687,3875.738,2875.738


In [34]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    ind_set_index=6,
    dos_index=6,
    plot_results=True,
    logger_bool=False,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 30,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 60.0,
    rsi_length = 14,
)
